In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tf_keras

2024-10-16 11:40:18.291855: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-16 11:40:18.424334: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-16 11:40:18.560038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 11:40:18.741191: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 11:40:18.798273: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 11:40:19.098162: I tensorflow/core/platform/cpu_feature_gu

In [5]:
train_data,validation_data,test_data=tfds.load(name="imdb_reviews",as_supervised=True,split=('train[:60%]','train[60%:]','test'))

2024-10-16 11:42:10.026960: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 100%|██████████| 1/1 [01:10<00:00, 70.83s/ url]
                                                                        

Dataset imdb_reviews downloaded and prepared to /home/codespace/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [6]:
train_example_batch,train_lable__batch=next(iter(train_data.batch(10)))

2024-10-16 11:43:48.085927: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [7]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [8]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[],
                           dtype=tf.string, trainable=True)

In [9]:
model=tf_keras.Sequential()
model.add(hub_layer)
model.add(tf_keras.layers.Dense(16,activation='relu'))
model.add(tf_keras.layers.Dense(1))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [12]:
history=model.fit(train_data.shuffle(20000).batch(100),epochs=20,
                  validation_data=validation_data.batch(100),verbose=1)

Epoch 1/20
150/150 [==============================] - 3s 13ms/step - loss: 0.8320 - accuracy: 0.5549 - val_loss: 0.6123 - val_accuracy: 0.6277
Epoch 2/20
150/150 [==============================] - 2s 10ms/step - loss: 0.5457 - accuracy: 0.7031 - val_loss: 0.5035 - val_accuracy: 0.7271
Epoch 3/20
150/150 [==============================] - 2s 11ms/step - loss: 0.4366 - accuracy: 0.7889 - val_loss: 0.4116 - val_accuracy: 0.8015
Epoch 4/20
150/150 [==============================] - 2s 10ms/step - loss: 0.3493 - accuracy: 0.8437 - val_loss: 0.3651 - val_accuracy: 0.8470
Epoch 5/20
150/150 [==============================] - 2s 10ms/step - loss: 0.2909 - accuracy: 0.8754 - val_loss: 0.3282 - val_accuracy: 0.8539
Epoch 6/20
150/150 [==============================] - 2s 10ms/step - loss: 0.2487 - accuracy: 0.8983 - val_loss: 0.3105 - val_accuracy: 0.8651
Epoch 7/20
150/150 [==============================] - 2s 11ms/step - loss: 0.2154 - accuracy: 0.9143 - val_loss: 0.3003 - val_accuracy: 0.8699

In [13]:
results=model.evaluate(test_data.batch(100),verbose=2)

250/250 - 2s - loss: 0.5148 - accuracy: 0.8483 - 2s/epoch - 8ms/step
